# ERC-4626: scanning historical price and performance of vaults

In [1]:
import os

os.environ["JSON_RPC_URL"] = "https://mainnet.infura.io/v3/6ec2c1604e2d477f9c9f7ea52aa8e0f3"

In [2]:
"""Scan historical vault share prices and fees.

- Scan prices for all vaults discovered earlier with ``scan-vaults.py``
- Write results to the Parquet file that is shared across all chains

Usage:

.. code-block:: shell

    export JSON_RPC_URL=...
    python scripts/erc-4626/scan-prices.py

Or for faster small sample scan limit the end block:

    END_BLOCK=5555721 python scripts/erc-4626/scan-prices.py

"""

import logging
import os
import pickle
import sys
from pathlib import Path
from pprint import pformat
from urllib.parse import urlparse

from eth_defi.chain import get_chain_name
from eth_defi.erc_4626.classification import create_vault_instance
from eth_defi.erc_4626.core import ERC4262VaultDetection
from eth_defi.provider.multi_provider import create_multi_provider_web3, MultiProviderWeb3Factory
from eth_defi.token import TokenDiskCache
from eth_defi.utils import setup_console_logging
from eth_defi.vault.historical import scan_historical_prices_to_parquet

logger = logging.getLogger(__name__)

# Read JSON_RPC_CONFIGURATION from the environment
JSON_RPC_URL = os.environ.get("JSON_RPC_URL")
if JSON_RPC_URL is None:
    try:
        urlparse(JSON_RPC_URL)
    except ValueError as e:
        raise ValueError(f"Invalid JSON_RPC URL: {JSON_RPC_URL}") from e


def main():
    setup_console_logging()

    token_cache = TokenDiskCache()

    # How many CPUs / subprocess we use
    max_workers = 1
    # max_workers = 1  # To debug, set workers to 1

    web3 = create_multi_provider_web3(JSON_RPC_URL)
    web3factory = MultiProviderWeb3Factory(JSON_RPC_URL, retries=5)
    name = get_chain_name(web3.eth.chain_id)

    min_deposit_threshold = 5

    start_block = 1
    end_block = os.environ.get("END_BLOCK")
    if end_block is None:
        end_block = web3.eth.block_number
    else:
        end_block = int(end_block)

    chain_id = web3.eth.chain_id

    # output_folder = os.environ.get("OUTPUT_FOLDER")
    # if output_folder is None:
    #     output_folder = Path("~/.tradingstrategy/vaults").expanduser()
    # else:
    #     output_folder = Path(output_folder).expanduser()

    # vault_db_fname = Path(f"{output_folder}/vault-db.pickle")
    # price_parquet_fname = output_folder / "vault-prices.parquet"

    output_folder = os.environ.get("OUTPUT_FOLDER")
    if output_folder is None:
        output_folder = Path("/Users/olaoluwatunmise/stablecoin_vaults/database").expanduser()
    else:
        output_folder = Path(output_folder).expanduser()
    
    vault_db_fname = Path(f"{output_folder}/vault-db.pickle")
    price_parquet_fname = output_folder / "vault-prices.parquet"

    os.makedirs(output_folder, exist_ok=True)


    print(f"Scanning vault historical prices on chain {web3.eth.chain_id}: {name}")

    assert vault_db_fname.exists(), f"File {vault_db_fname} does not exist - run scan-vaults.py first"

    vault_db = pickle.load(vault_db_fname.open("rb"))

    chain_vaults = [v for v in vault_db.values() if v["_detection_data"].chain == chain_id]
    print(f"Chain {name} has {len(chain_vaults):,} vaults in the vault detection database")

    if len(chain_vaults) == 0:
        print(f"No vaults on chain {name}")
        sys.exit(0)

    vaults = []
    #start = 999_999_999_999
    start = end_block - 100  # Smaller time range

    for row in chain_vaults:
        detection: ERC4262VaultDetection
        detection = row["_detection_data"]
        address = detection.address

        if detection.deposit_count < min_deposit_threshold:
            # print(f"Vault does not have enough deposits: {address}, has: {detection.deposit_count}, threshold {min_deposit_threshold}")
            continue

        vault = create_vault_instance(web3, address, detection.features, token_cache=token_cache)
        if vault is not None:
            vault.first_seen_at_block = detection.first_seen_at_block
            vaults.append(vault)
            start = min(start, detection.first_seen_at_block)
        else:
            # print(f"Vault does not have a supported reader: {address}")
            pass

         # Or reduce number of vaults
        vaults = vaults[:10]  # Only first 10 vaults
        #chunk_size = 8  

    print(f"After filtering vaults for non-interesting entries, we have {len(vaults):,} vaults left")

    scan_result = scan_historical_prices_to_parquet(
        output_fname=price_parquet_fname,
        web3=web3,
        web3factory=web3factory,
        vaults=vaults,
        start_block=start,
        end_block=end_block,
        max_workers=max_workers,
        chunk_size=32,
        token_cache=token_cache,
    )

    token_cache.commit()
    print(f"Token cache size is {token_cache.get_file_size():,} bytes, {len(token_cache):,} tokens")
    print("Scan complete")
    print(pformat(scan_result))


if __name__ == "__main__":
    main()

Scanning vault historical prices on chain 1: Ethereum
Chain Ethereum has 100 vaults in the vault detection database
After filtering vaults for non-interesting entries, we have 10 vaults left


Loading token metadata for 10 addresses using 8 workers:   0%|          | 0/1 [00:00<?, ?it/s]

Preparing historical multicalls for 10 readers using 1 workers:   0%|          | 0/10 [00:00<?, ? readers/s]

Reading historical vault price data for chain 1 with 1 workers, blocks 14,521,099 - 22,931,011:   0%|         …

/Users/olaoluwatunmise/stablecoin_vaults/vaults/lib/python3.10/site-packages/joblib/parallel.py:1383: UserWarning: The backend class 'SequentialBackend' does not support timeout. You have set 'timeout=1800' in Parallel but the 'timeout' parameter will not be used.
  warnings.warn(


Token cache size is 90,112 bytes, 279 tokens
Scan complete
{'chain_id': 1,
 'chunks_done': 359,
 'existing': True,
 'existing_row_count': 0,
 'file_size': 976002,
 'output_fname': PosixPath('/Users/olaoluwatunmise/stablecoin_vaults/database/vault-prices.parquet'),
 'rows_deleted': 11439,
 'rows_written': 11469}


### Load the price data

In [3]:
import pandas as pd

# Load the price data
prices_df = pd.read_parquet('/Users/olaoluwatunmise/stablecoin_vaults/database/vault-prices.parquet')
print(f"Loaded {len(prices_df)} price records")
print(prices_df.head())

Loaded 11469 price records
   chain                                     address  block_number  \
0      1  0x48f8d7943899d9b4f34ccb5ba1b92695433226e8      14521099   
1      1  0x48f8d7943899d9b4f34ccb5ba1b92695433226e8      14528299   
2      1  0x48f8d7943899d9b4f34ccb5ba1b92695433226e8      14535499   
3      1  0xb78ebb2248bb72380e690246f9631cf58c07b444      14535499   
4      1  0xa066620554af0d50703e5ee3c58c022c6ae5339d      14535499   

            timestamp  share_price  total_assets  total_supply  \
0 2022-04-04 18:17:15          1.0      0.300000      0.300000   
1 2022-04-05 21:17:13          1.0      0.300000      0.300000   
2 2022-04-07 00:16:14          1.0      0.357827      0.357827   
3 2022-04-07 00:16:14          1.0      0.050000      0.050000   
4 2022-04-07 00:16:14          1.0      0.050000      0.050000   

   performance_fee  management_fee errors  
0              NaN             NaN         
1              NaN             NaN         
2              NaN     